In [15]:
import import_ipynb
import numpy as np
import pandas as pd

#import stat_functions

In [2]:
filepath = "C:/Users/trfit/OneDrive/Documents/Hobbies/CRG/stats 2025/prepped_data/" 
file = "VL_Akron_data.csv"
df = pd.read_csv(filepath+file)


In [19]:
filepath = "C:/Users/trfit/OneDrive/Documents/Hobbies/CRG/stats 2025/prepped_data/" 
VL_files = [
    "VL_Akron_data.csv",
    "VL_Athens_data.csv",
    "VL_Terrorz_data.csv"]
BS_files = [
    "BS_Akron_data.csv",
    "BS_Athens_data.csv",
    "BS_Great_Lakes_data.csv",
    "BS_Louisville_data.csv"]
df_lst = []
for file in VL_files:
    df_lst.append(pd.read_csv(filepath+file))
df = pd.concat(df_lst, ignore_index=True)

In [20]:
df

,OP,Half,Jam,noPivot,Jammer,Jammer_Box_1,Jammer_Box_2,Jammer_Box_3,Pivot,Pivot_Box_1,...,SP_Game Total,SP_Trips,SP_OP_LOST,SP_OP_LEAD,SP_OP_CALL,SP_OP_INJ.,SP_OP_NI,SP_OP_Jam Total,SP_OP_Game Total,SP_OP_Trips
0,Akron,A,1,NaN,517,NaN,NaN,NaN,721,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Akron,B,1,NaN,27,NaN,NaN,NaN,100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Akron,A,2,NaN,27,NaN,NaN,NaN,721,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Akron,B,2,NaN,517,NaN,NaN,NaN,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Akron,A,3,NaN,721,NaN,NaN,NaN,6,NaN,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Terrorz,A,19,NaN,16,NaN,NaN,NaN,88,$,...,120.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,35.0,1.0
128,Terrorz,A,2,NaN,27,NaN,NaN,NaN,6,NaN,...,19.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,4.0,1.0
129,Terrorz,A,3,NaN,16,NaN,NaN,NaN,88,NaN,...,23.0,2.0,NaN,NaN,NaN,NaN,NaN,0.0,8.0,0.0
130,Terrorz,A,6,NaN,27,NaN,NaN,NaN,88,NaN,...,43.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,11.0,1.0


# Feature preparation

Jam Embeddings: | Jam_id | point_diff | lead | Trips | jammer_penalty_counter | blocker_penalty_counter | skater1_as_jammer |...| skatern_as_jammer | skater1_as_blocker |...| skatern_as_blocker |

curr_col = ['OP','Half','Jam',
            'LEAD', #'OP_LEAD',
            'Jammer', 'Pivot', 'Blocker_1', 'Blocker_2', 'Blocker_3',
            'Trips','OP_Trips', 
            'Jam Total', 'Game Total','OP_Jam Total', 'OP_Game Total',
            'Jammer_Box_1', 'Jammer_Box_2','Jammer_Box_3',
            'Pivot_Box_1','Pivot_Box_2','Pivot_Box_3',
            'Blocker_1_Box_1','Blocker_1_Box_2','Blocker_1_Box_3',
            'Blocker_2_Box_1','Blocker_2_Box_2','Blocker_2_Box_3',
            'Blocker_3_Box_1','Blocker_3_Box_2','Blocker_3_Box_3']

Ready: 
 - game_id: OP
 - points_for: Jam Total, Game Total
 - lead_for_us (0/1): boolean
 - our_jammer (string)
 - jam_duration: Trips
 - points_against: OP_Jam Total, OP_Game Total

Need work:
- jam_id: need to separate by game & by half
- our_blockers (list/JSON/string of 4 skater IDs): separate columns, int type

Dont have
 - penalties_our_team (int)
 - penalties_their_team (int)


box characters: 
- –  = Entered the box mid-jam.
- S = Entered between jams or continued from previous jam.
- | = Finished service.
- '+' combines - & |
- 'dollar sign' Combines + and S. Almost always paired with a - in the previous jam
- 3 = Skater injured or needed sub, sits out 3 jams.

In [21]:
# Functions

def build_encoded_matrix(df, cols, skaters):
    # This function calculates the average normalized jam +/- for each skater using matrix multiplication. This method is used because it makes 
    # it easier to remove a skater and recalculate.

    df_encoded = pd.get_dummies(df, columns=cols)

    # Combine the position boolean columns, so each skater has one boolean column, True = skater was in the jam, False = skater was not in the jam
    #cols = ['Jammer', 'Pivot', 'Blocker 1', 'Blocker 2', 'Blocker 3']
    for skater in skaters:
        cols_comb = []
        for col in cols:
            if col+"_"+str(skater) in df_encoded.columns:
                cols_comb.append(col+"_"+str(skater))
        df_encoded[str(skater)] = df_encoded[cols_comb].any(axis='columns')
    
        for col in cols_comb:
            df_encoded = df_encoded.drop(col, axis=1)

    return df_encoded

In [31]:
# Create feature df
# Jam Embeddings: | x jam_id | x point_diff | x lead | x trips | x jammer_penalty_counter | x blocker_penalty_counter | skater1_as_jammer |...| skatern_as_jammer | skater1_as_blocker |...| skatern_as_blocker |
X_col = []
df_features = pd.DataFrame()

# Make Jam_id
for col in ['OP', 'Half',  'Jam']:
    df[col] = df[col].astype("string")
df_features['jam_id'] = df['OP'] +"_"+ df['Half'] +"_"+ df['Jam']

# Make other features
df_features['point_diff'] = df['Jam Total'] - df['OP_Jam Total']
df_features['lead'] = df['LEAD'].replace({np.nan: False, "X": True}).infer_objects(copy=False)
df_features['trips'] = df['Trips']
X_col += ['lead', 'trips']

# Make jammer_penalty_counter
for col in ['Jammer_Box_1', 'Jammer_Box_2', 'Jammer_Box_3']:
    df[col] = df[col].replace({np.nan: 0, '+': 1, '-': 1, '$': 1}).infer_objects(copy=False)
df_features['jammer_penalty_counter'] = df['Jammer_Box_1'] + df['Jammer_Box_2'] + df['Jammer_Box_3']

df_features['blocker_penalty_counter'] = 0
for col in ['Pivot_Box_1','Pivot_Box_2','Pivot_Box_3',
            'Blocker_1_Box_1','Blocker_1_Box_2','Blocker_1_Box_3',
            'Blocker_2_Box_1','Blocker_2_Box_2','Blocker_2_Box_3',
            'Blocker_3_Box_1','Blocker_3_Box_2','Blocker_3_Box_3']:
    #print(col)
    #print(df[col].dtype)
    #print(df[col].unique())
    df[col] = df[col].replace({np.nan: 0, '+': 1, '-': 1, '$': 1, 'S': 1, '3':0}).infer_objects(copy=False)
    df_features['blocker_penalty_counter'] += df[col]
X_col += ['jammer_penalty_counter', 'blocker_penalty_counter']    

# jammer & blocker one hot encoding
blocker_cols = ['Pivot','Blocker_1', 'Blocker_2', 'Blocker_3']
df_encoded = build_encoded_matrix(
    df[blocker_cols], blocker_cols, 
    pd.concat([df['Pivot'],df['Blocker_1'],df['Blocker_2'],df['Blocker_3']]).unique()
)
X_col += [col for col in df_encoded.columns]
df_features = pd.concat([df_features, df_encoded], axis=1)

df_encoded = build_encoded_matrix(
    df['Jammer'], ['Jammer'], 
    df['Jammer'].unique()
)

for jammer in df['Jammer'].unique():
    df_features[str(jammer)+"_jammer"] = False
    df_features[str(jammer)+"_jammer"] = df_features[str(jammer)+"_jammer"] | df_encoded[jammer]
    X_col.append(str(jammer)+"_jammer")


Pivot_Box_1
int64
[0 1]
Pivot_Box_2
int64
[0 1]
Pivot_Box_3
float64
[0.]
Blocker_1_Box_1
int64
[0 1]
Blocker_1_Box_2
float64
[0.]
Blocker_1_Box_3
float64
[0.]
Blocker_2_Box_1
int64
[0 1]
Blocker_2_Box_2
float64
[0.]
Blocker_2_Box_3
float64
[0.]
Blocker_3_Box_1
int64
[0 1]
Blocker_3_Box_2
int64
[0 1]
Blocker_3_Box_3
float64
[0.]


C:\Users\trfit\AppData\Local\Temp\ipykernel_28048\1191719805.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_features['lead'] = df['LEAD'].replace({np.nan: False, "X": True}).infer_objects(copy=False)


In [33]:
train_set = df_features.sample(frac=0.8, random_state=42)

# Dropping all those indexes from the dataframe that exists in the train_set
test_set = df_features.drop(train_set.index)
train_set.shape, test_set.shape

((106, 39), (26, 39))

In [34]:
from sklearn.linear_model import Ridge 
alpha = 20.0 # you can tune this 

In [35]:
ridge_model = Ridge(alpha=alpha)

In [36]:
X = train_set[X_col]
y = train_set['point_diff'] 

In [37]:
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.linear_model import Ridge
#from sklearn.pipeline import Pipeline
#from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, KFold
#from sklearn.metrics import mean_squared_error, r2_score

In [38]:

# Cross-validation to gauge performance 
cv = KFold(n_splits=5, shuffle=True, random_state=42) 

neg_mse_scores = cross_val_score( ridge_model, X, y, cv=cv, scoring="neg_mean_squared_error" ) 

rmse_scores = np.sqrt(-neg_mse_scores) 

r2_scores = cross_val_score(ridge_model, X, y, cv=cv, scoring="r2") 
print("CV RMSE:", rmse_scores.mean(), "+/-", rmse_scores.std()) 
print("CV R^2:", r2_scores.mean(), "+/-", r2_scores.std()) 

#Fit the final model on all data:
ridge_model.fit(X, y) 


CV RMSE: 4.19644893146648 +/- 0.65134058139965
CV R^2: 0.6154844884282933 +/- 0.06647893525221824


Ridge(alpha=20.0)

In [8]:
ridge_model.fit(X, y)

Ridge(alpha=20.0)

In [14]:
ridge_model.coef_ # ndarray of shape (n_features,) or (n_targets, n_features)Weight vector(s).

ridge_model.intercept_ # float or ndarray of shape (n_targets,)Independent term in decision function. Set to 0.0 if fit_intercept = False.

ridge_model.n_iter_ # _None or ndarray of shape (n_targets,) Actual number of iterations for each target. Available only for sag and lsqr solvers. Other solvers will return None.

ridge_model.n_features_in_ # int Number of features seen during fit.

ridge_model.feature_names_in_ # ndarray of shape (n_features_in_,)

array(['lead', 'trips', 'jammer_penalty_counter',
       'blocker_penalty_counter', '721', '6', '13', '711', '222', '100',
       '27', '555', '517', '603', '800', '35', '36', '83', '48',
       '517_jammer', '27_jammer', '721_jammer', '100_jammer',
       '555_jammer', '6_jammer', '711_jammer'], dtype=object)

In [47]:
y_test = ridge_model.predict(test_set[X_col])

In [55]:
n = 0
for i in test_set['point_diff']:
    print(i,round(y_test[n]), round(i - y_test[n]))
    n+=1

-4 -1 -3
-4 -1 -3
-4 -5 1
-10 -5 -5
4 7 -3
2 1 1
0 2 -2
0 7 -7
-3 1 -4
0 -3 3
12 7 5
4 4 0
6 2 4
8 5 3
4 3 1
-7 6 -13
0 1 -1
3 1 2
0 2 -2
19 16 3
3 2 1
8 5 3
5 3 2
4 4 0
-8 -8 0
20 14 6


In [48]:
y_test - test_set['point_diff']

1       2.563072
2       3.156973
14     -0.797438
20      5.371065
21      3.432576
23     -1.170971
29      1.646635
37      6.743433
52      3.885475
60     -3.340435
71     -4.690858
74      0.187452
82     -4.414322
86     -3.170858
87     -1.291002
92     12.758204
99      1.093400
102    -1.752781
103     2.146342
106    -3.286388
107    -1.235190
116    -3.054626
118    -1.831389
120     0.082411
121     0.375196
131    -5.652241
Name: point_diff, dtype: float64

In [27]:
coef_series = pd.Series( ridge_model.coef_, index=X_col ).sort_values(ascending=False) #coef_series.head(20), coef_series.tail(20)
coef_series #

trips                      1.972550
lead                       1.268568
711                        0.545353
27_jammer                  0.487598
100_jammer                 0.466506
603                        0.382082
517_jammer                 0.356210
36                         0.253502
222                        0.239666
13                         0.112763
27                         0.081897
555_jammer                -0.006516
48                        -0.034897
721_jammer                -0.042636
6                         -0.084088
100                       -0.113796
517                       -0.122891
83                        -0.151202
555                       -0.197479
35                        -0.244260
800                       -0.277977
721                       -0.388674
6_jammer                  -0.431254
blocker_penalty_counter   -0.490507
711_jammer                -0.829908
jammer_penalty_counter    -1.040142
dtype: float64

In [39]:
coef_series = pd.Series( ridge_model.coef_, index=X_col ).sort_values(ascending=False) #coef_series.head(20), coef_series.tail(20)
coef_series #

trips                      2.829784
lead                       2.291134
27_jammer                  0.991166
88                         0.791936
33                         0.633137
71                         0.500029
933                        0.409031
603                        0.390771
36                         0.344039
100_jammer                 0.313906
711                        0.249547
423                        0.174729
16_jammer                  0.162980
48                         0.147556
603_jammer                 0.126077
16                         0.124145
33_jammer                  0.107978
83                         0.086542
555_jammer                 0.055629
6_jammer                   0.038914
800                       -0.133060
13                        -0.192342
137                       -0.211925
88_jammer                 -0.280083
721_jammer                -0.291597
27                        -0.295412
517                       -0.305007
517_jammer                -0

In [28]:
jammer_effects = coef_series[[c for c in coef_series.index if c.endswith("_jammer")]] 

In [30]:
jammer_effects

27_jammer     0.487598
100_jammer    0.466506
517_jammer    0.356210
555_jammer   -0.006516
721_jammer   -0.042636
6_jammer     -0.431254
711_jammer   -0.829908
dtype: float64

In [44]:
df_test

,lead,trips,jammer_penalty_counter,blocker_penalty_counter,721,6,13,711,222,100,...,36,83,48,517_jammer,27_jammer,721_jammer,100_jammer,555_jammer,6_jammer,711_jammer
0,0,0,0,0,True,0,0,0,0,True,...,True,0,0,True,0,0,0,0,0,0


In [33]:
len(X.columns)

26

np.float64(-4.6801483238309896)

In [48]:
df_test = pd.DataFrame([[0]*len(X_col)], columns=X_col)
for col in ['517_jammer', '721', '603', '36', '100']:
    df_test[col] = True
ridge_model.predict(df_test)[0] 

np.float64(-4.6801483238309896)

In [49]:
df_test = pd.DataFrame([[0]*len(X_col)], columns=X_col)
for col in ['517_jammer', '711', '222', '83', '13']:
    df_test[col] = True
ridge_model.predict(df_test)[0] 

np.float64(-4.06668275540865)

In [51]:
df_features.head(10)

,jam_id,point_diff,lead,trips,jammer_penalty_counter,blocker_penalty_counter,721,6,13,711,...,36,83,48,517_jammer,27_jammer,721_jammer,100_jammer,555_jammer,6_jammer,711_jammer
0,Akron_A_1,-4,False,0,0.0,0.0,True,False,False,False,...,True,False,False,True,False,False,False,False,False,False
1,Akron_A_2,-4,False,1,0.0,0.0,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,Akron_A_3,-7,False,0,0.0,0.0,False,True,False,False,...,True,False,True,False,False,True,False,False,False,False
3,Akron_A_4,-3,False,2,0.0,0.0,False,False,True,False,...,False,True,False,True,False,False,False,False,False,False
4,Akron_A_5,4,False,2,1.0,1.0,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
5,Akron_A_6,3,True,1,0.0,1.0,False,False,False,True,...,True,False,False,False,False,True,False,False,False,False
6,Akron_A_7,-8,False,0,1.0,1.0,False,False,True,True,...,False,True,False,True,False,False,False,False,False,False
7,Akron_A_8,-4,False,1,1.0,1.0,False,True,False,True,...,True,False,True,True,False,False,False,False,False,False
8,Akron_A_9,0,False,0,0.0,0.0,False,False,True,False,...,False,True,False,False,False,False,True,False,False,False
9,Akron_A_10,3,True,1,0.0,1.0,False,True,False,False,...,True,False,True,False,True,False,False,False,False,False


['Akron']
114
['Athens']
114
['Terrorz']
114
['Akron']
114
['Athens']
114
['Great_Lakes']
114
['Louisville']
114
